
# Why We Are Doing This

## Rationale

Worker fatigue is a pervasive issue in industrial settings like car assembly lines, often leading to decreased productivity, increased error rates, and a heightened risk of workplace accidents. Traditional methods of task rotation and scheduling often lack the precision to account for varying fatigue levels associated with different tasks. As a result, workers may experience inconsistent fatigue levels, affecting their well-being and work output.

## Objectives

By solving this problem, we aim to create a safer, more efficient workplace environment that respects the physical limitations of the workforce while optimizing for productivity.


## Problem Statement


The objective of this notebook is to develop a quantitative model to manage worker fatigue in a car assembly process. We aim to optimize work rotations based on a mathematical model of fatigue accumulation over time. 

The model employs an exponential fatigue function, $$ f(t) = 1 - \exp(-\lambda t) $$

to describe how fatigue builds up for workers engaged in different types of tasks, categorized as Red, Yellow, and Green based on their ergonomic ratings.

Through this model, we aim to:

1. Calculate the rate parameter $ \lambda $  for each ergonomic rating based on given time estimates to reach a fatigue level of 0.80.
2. Validate these $ \lambda $  estimates using real-world data or expert opinions.
3. Incorporate these $ \lambda $  values into an optimization model to minimize the overall fatigue level across workers while considering constraints like worker availability and task sequencing.


## Background 


### Theoretical Framework for Fatigue Modeling in Sub-tasks:

1. **Hierarchy of Tasks and Sub-tasks**: In this setup, each "job" is a collection of sub-tasks. Each sub-task has its own fatigue rating, which contributes to the overall fatigue for the job. This recognizes the reality that not all parts of a job contribute equally to worker fatigue.

| Sub-Task ID | Description                                                                  | Ergonomic Rating |
|-------------|------------------------------------------------------------------------------|------------------|
| Sub-Task 1  | READ BODY ASSEMBLY SHEET                                                     | Green            |
| Sub-Task 2  | VISUALLY INSPECT CARPET LABEL PART MARK MATCHES BAS.                         | Yellow           |
| Sub-Task 3  | TIGHTEN NUT PAINT CUTTING 8MM (X3) TO ACCELERATOR PEDAL AND DASHBOARD STUDS  | Red              |
| Sub-Task 4  | CONFIRM CORRECT PART, SET BRAKE PEDAL TO MASTER POWER STUDS                  | Green            |
| Sub-Task 5  | SET BRAKE PEDAL PIN TO MASTER POWER YOKE AND BRAKE PEDAL                     | Yellow           |
| Sub-Task 6  | SET LOCK PIN 8MM TO BRAKE PEDAL PIN                                         | Red              |
| Sub-Task 7  | PULL CHECK BRAKE PEDAL TO CONFIRM BRAKE PEDAL PIN INSTALL CONDITION         | Yellow           |
| Sub-Task 8  | TIGHTEN NUT FLANGE 8MM (X4) TO BRAKE PEDAL AND MASTER POWER                  | Green            |



2. **Time-Dependent Fatigue**: The concept of time-dependent fatigue would still apply, but now it would be even more granular. Instead of a single time to reach a fatigue level for a whole job, each sub-task could contribute differently to reaching that fatigue threshold. This would involve integrating over time spent on different sub-tasks.

3. **Multi-dimensional Fatigue**: Since each sub-task can be different in nature (e.g., some might be physically demanding while others require intense concentration), you could consider multi-dimensional fatigue, where each dimension corresponds to a different type of fatigue (e.g., physical, mental).

4. **State-Dependent Effects**: The fatigue from a sub-task could depend not just on the sub-task itself but also on the state of the worker at the time they perform it. For example, doing a physically demanding sub-task might be more tiring if it follows a mentally exhausting one, even if the two sub-tasks are part of different jobs.

5. **Dynamic Recovery**: Introducing the concept of dynamic recovery rates based on the type of sub-task could make the model more realistic. For example, a less demanding sub-task could serve as a "recovery period" after a more demanding one.

6. **Constraints and Boundaries**: While maximizing worker efficiency by minimizing fatigue, constraints such as worker skills, sub-task prerequisites, and maximum allowable fatigue would play a significant role in the model. 

7. **Optimization Criteria**: The ultimate goal would still be to find the allocation of sub-tasks to workers that minimizes overall fatigue, subject to constraints. However, this could be a multi-objective optimization problem if you're also considering other criteria like job completion time or quality.


The fatigue function $$ f(t) = 1 - \exp(-\lambda t) $$ is a classic example of an exponential decay model, 
which is widely used to describe processes where something decreases at a rate proportional to its current value. 
In this case, the function models how fatigue accumulates over time $( t )$ with a rate parameter $( \lambda )$

### Key Features of the Function:

1. **Initial Condition**: $ f(0) = 0 $ implies that there's no fatigue at the beginning $ t=0 $.
  
2. **Asymptotic Behavior**: As $( t )$ approaches infinity, $ f(t) $ approaches 1, indicating a maximum fatigue level of 1.

3. **Rate of Fatigue**: The parameter $ \lambda $ controls the rate of fatigue accumulation. Higher $ \lambda $ means faster fatigue accumulation.

4. **Accumulative and Time-Dependent**: The function models the fatigue as an accumulative measure that increases with time spent on a task or sub-task.

### How It Fits into the Model:

1. **Per Sub-Task Fatigue**: For each sub-task, you could calculate the expected fatigue given its duration and the associated $ \lambda $ value. 

2. **Overall Fatigue**: The total fatigue for a sequence of sub-tasks would be the sum of the individual fatigues. This assumes that fatigue from different sub-tasks is additive, which is a simplification but may be reasonable for a first-pass model.

3. **Constraint**: The constraint $f(t) \leq 0.80  $ could be used to ensure that no worker crosses the maximum allowable fatigue. This allows you to align with the workplace safety standards or guidelines.

4. **Optimization**: Your objective function would likely be a sum of $ f(t) $ terms for each worker and each sub-task they complete, with the goal of minimizing this sum. Constraints could include worker availability, required sub-task sequencing, etc.

5. **Dynamic Adjustment**: Given that $ f(t) $ is time-dependent, you could also look at dynamic models that adjust the work rotation in real-time to minimize fatigue.

## Generating Fatigue Rate $ \lambda $ 

Generating Fatigue Rate $ \lambda $  estimates  is crucial for the model's accuracy and effectiveness. 

Here are some ways of the method that I considered

### 1. Real-World Testing: - (REJECTED: Not Enough Resources) 

- **Design**: Conduct controlled trials where workers perform Red, Yellow, and Green tasks for extended periods.
- **Metrics**: Collect data on performance, error rates, and physiological markers (like heart rate) over time.
- **Analysis**: Check if performance declines or error rates increase around the estimated times.

### 2. Expert Reviews: (Accepted: Leverage Existing Ergonomic Specialist Knowledge)

See explaination below 

- **Consult**: Involve ergonomics experts to review these initial estimates along with the types of tasks involved.
- **Adjust**: Use their feedback to refine the estimates.


### 3. Historical Data Analysis (REJECTED: Data Unavailable) :

- **Data**: Use past records of worker performance and fatigue-related incidents.
- **Comparison**: See if the data shows a spike in incidents or a decline in performance around the estimated times.

### 4. Surveys and Self-Assessments (REJECTED: Unreliable):

- **Method**: Have workers rate their fatigue levels after performing these tasks for varying durations.
- **Analysis**: Look for a significant uptick in self-reported fatigue levels around the estimated times.


## Generating $ \lambda $ Using Expert Reviews

We set 0.80 fatigue level as maximum allowable fatigue and define the maxiumum minutes for each green, yellow, red task

In occupational ergonomics and human factors research, fatigue is often categorized based on its impact on performance, physiological indicators, and subjective experiences. To quantify a fatigue level of 0.8 within a pre-existing framework, we can look at it through the lens of the "Rate of Perceived Exertion (RPE)" scale, which is widely used in sports science and ergonomics. 

The RPE scale ranges from 6 to 20, where 6 indicates "no exertion at all," and 20 indicates "maximal exertion." We could map a fatigue level of 0.8 to an RPE value by scaling it proportionally. This would equate to an RPE value of \( 0.8 \times (20 - 6) + 6 = 17.2 \), which corresponds to a level described as "very hard" on the RPE scale. 

In this context, a fatigue level of 0.8 could be interpreted as:

- **Performance Impact**: Significant decrease in work performance, increased error rates, and reduced reaction times.
- **Physiological Indicators**: Elevated heart rates, increased respiration, and visible signs of muscle fatigue.
- **Subjective Experience**: Workers would likely report feeling "very tired" or "nearing exhaustion," potentially impacting both mental and physical faculties.

Therefore, a fatigue level of 0.8 would be a critical threshold, indicating that a worker is operating at a level that is sustainable only for a short duration and is at elevated risk for errors or injuries. Special care should be taken to rotate tasks or provide sufficient rest when this level of fatigue is approached.

#### Context-Specific Examples:

- **Green Tasks (540 minutes)**: Tasks like walking and standing for visual inspection are considered low fatigue. Check if workers can generally perform these tasks for approximately 540 minutes (9 hours) without significant performance decline or discomfort.
  
- **Yellow Tasks (300 minutes)**: Tasks like tightening bolts while standing up might be moderately fatiguing. Observe if workers begin to make more errors or slow down after about 300 minutes (5 hours).
  
- **Red Tasks (180 minutes)**: Overhead work with poor posture is highly fatiguing. Monitor for signs of strain or discomfort after around 180 minutes (3 hours).

### CODE

In [2]:
from scipy.optimize import root_scalar
import numpy as np

# Define the target fatigue level and times for Red, Yellow, Green units
target_fatigue = 0.80
times_to_reach_fatigue = {'Red': 180, 'Yellow': 300, 'Green': 540}

# Define the function to find the root of
def equation_to_solve(lambda_value, time):
    return 1 - np.exp(-lambda_value * time) - target_fatigue

# Initialize a dictionary to store the lambda values for each unit type
lambda_values = {}

# Solve for lambda for each unit type
for unit_type, time in times_to_reach_fatigue.items():
    result = root_scalar(equation_to_solve, args=(time,), bracket=[0, 1], method='brentq')
    lambda_values[unit_type] = result.root

lambda_values

{'Red': 0.00894132173526088,
 'Yellow': 0.005364793041447002,
 'Green': 0.0029804405785820516}

Red_$ \lambda $  = 0.00894

Yellow_$ \lambda $ = 0.00536

Green_$ \lambda $ = 0.00298

$ \lambda $  values indicate the rate of fatigue accumulation for tasks with different ergonomic ratings. 

***For example, a Red_$ \lambda $ task will accumulate fatigue at a rate of approximately 0.00894 per minute, according to the exponential fatigue model.***

In [7]:
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.optimize import linear_sum_assignment

# Generate sample data for 6 jobs and their total fatigue scores
job_data = {
    'Job_ID': [f'Job_{i+1}' for i in range(6)],
    'Total_Fatigue_Score': np.random.randint(50, 100, size=6)
}
job_df = pd.DataFrame(job_data)

# Generate sample data for 6 workers with worker knowledge of 4 jobs
worker_data = {
    'Worker_ID': [f'Worker_{i+1}' for i in range(6)],
    'Known_Jobs': [','.join(np.random.choice(job_df['Job_ID'], size=4, replace=False)) for _ in range(6)]
}
worker_df = pd.DataFrame(worker_data)



In [16]:
# Simulating Fatigue Mapping For Each Job
job_df

,Job_ID,Total_Fatigue_Score
0,Job_1,98
1,Job_2,68
2,Job_3,55
3,Job_4,92
4,Job_5,93
5,Job_6,56


In [9]:
# Simulating Worker Knowledge
worker_df

,Worker_ID,Known_Jobs
0,Worker_1,"Job_6,Job_4,Job_3,Job_2"
1,Worker_2,"Job_4,Job_3,Job_2,Job_5"
2,Worker_3,"Job_3,Job_2,Job_5,Job_6"
3,Worker_4,"Job_4,Job_1,Job_2,Job_3"
4,Worker_5,"Job_3,Job_6,Job_1,Job_4"
5,Worker_6,"Job_6,Job_4,Job_2,Job_3"


In [13]:
# Since each worker should be assigned 4 jobs, we'll use combinatorial optimization 
# to find the best set of 4 jobs for each worker

# We'll calculate the total fatigue score for each possible combination of 4 jobs for each worker

# Create a new cost matrix to store the total fatigue score for each combination of 4 jobs for each worker
combinations_cost_matrix = {}

for i, row in worker_df.iterrows():
    worker_id = row['Worker_ID']
    known_jobs = row['Known_Jobs'].split(',')
    # Generate all combinations of 4 jobs that the worker knows
    job_combinations = combinations(known_jobs, 4)
    for job_comb in job_combinations:
        total_fatigue_score = sum(job_df[job_df['Job_ID'].isin(job_comb)]['Total_Fatigue_Score'])
        combinations_cost_matrix[(worker_id, job_comb)] = total_fatigue_score

# Now, we'll find the set of 4 jobs for each worker that minimizes the overall fatigue in the zone
# Since each worker can only perform 4 jobs, we'll assign the combination with the lowest 
# total fatigue score to each worker
optimized_schedule = {}
for worker_id in worker_df['Worker_ID']:
    # Find the combination of 4 jobs with the lowest total fatigue score for this worker
    best_combination = min({k: v for k, v in combinations_cost_matrix.items() if k[0] == worker_id}, key=lambda x: x[1])
    optimized_schedule[worker_id] = best_combination[1]

optimized_schedule_df = pd.DataFrame(list(optimized_schedule.items()), columns=['Worker_ID', 'Assigned_Jobs'])

# Calculate the total fatigue score for each individual based on their assigned jobs
optimized_schedule_df['Total_Fatigue_Score'] = optimized_schedule_df['Assigned_Jobs'].apply(
    lambda x: sum(job_df[job_df['Job_ID'].isin(x)]['Total_Fatigue_Score'])
)

total_fatigue_score_zone = optimized_schedule_df['Total_Fatigue_Score'].sum()

optimized_schedule_df['Total_Fatigue_Score'] = optimized_schedule_df['Total_Fatigue_Score']
optimized_schedule_df

,Worker_ID,Assigned_Jobs,Total_Fatigue_Score
0,Worker_1,"(Job_6, Job_4, Job_3, Job_2)",271
1,Worker_2,"(Job_4, Job_3, Job_2, Job_5)",308
2,Worker_3,"(Job_3, Job_2, Job_5, Job_6)",272
3,Worker_4,"(Job_4, Job_1, Job_2, Job_3)",313
4,Worker_5,"(Job_3, Job_6, Job_1, Job_4)",301
5,Worker_6,"(Job_6, Job_4, Job_2, Job_3)",271


In [15]:
total_fatigue_score_zone

1736

### Summary of Work Rotation Optimization for Minimizing Worker Fatigue

#### Objective
The primary objective of this project was to develop an optimized work rotation schedule that minimizes the overall fatigue within a work zone. The work zone consists of six workers, each with knowledge of 3 to 4 jobs, and six distinct jobs, each with its fatigue score based on ergonomic evaluations.

#### Methodology
1. **Data Collection**: The first step was gathering the fatigue scores for each job and the list of jobs that each worker is skilled in. The fatigue scores were calculated based on ergonomic evaluations.
  
2. **Fatigue Function**: The fatigue function  $ f(t) = 1 - \exp(-\lambda t) $ was used to model how fatigue accumulates over time for each worker. The parameter $ \lambda $ was calculated based on the time it would take for a worker to reach a fatigue level of 0.80 for Red, Yellow, and Green ergonomic ratings.

3. **Optimization**: We used combinatorial optimization techniques to assign the best combination of 4 jobs to each worker. The goal was to minimize the sum of the fatigue scores for each worker and, by extension, the entire work zone.

4. **Constraints**: Each worker was restricted to a set of jobs they are skilled in, and the total fatigue for each worker was not to exceed a pre-defined threshold.

5. **Result**: An optimized schedule was generated, which assigns a set of 4 jobs to each worker in a way that minimizes their total fatigue as well as the fatigue of the entire zone.

#### Key Takeaways
The project successfully provides a scientifically grounded, quantifiable approach to manage and optimize work rotations in a way that considers worker well-being and operational efficiency. This process can serve as a framework for similar optimization problems in various industrial settings.

# Thanks